---
title: The Hyperactive Spring 2024 Tornado Season
description: A detailed overview of the historic season.

  - Andrew Blackford[1], Trent Cowan[1], Udaysankar Nair[1], Josh Wurman[2], Karen Kosiba[2], David Roueche[3], Catherine Finley[4], and Jana Houser[5]

[1] The University of Alabama in Huntsville (UAH)

[2] Colorado State University

[3] Auburn University

[4] The University of North Dakota

[5] The Ohio State University
  - Andrew Blackford (editor)
date: August 7, 2025
execute:
   freeze: true
---

# Run This Notebook

<h3><a href="https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/stories/derechos.ipynb&branch=main">🚀 Launch in VEDA JupyterHub (requires access)</a></h3>

<h4> To obtain credentials to VEDA Hub, <a href = "https://docs.openveda.cloud/user-guide/scientific-computing/getting-access.html"> follow this link for more information.</a></h4>

<div class="alert alert-block" style="
     background-color: #f8d7da;
     color: #721c24;
     border-left: 4px solid #28a745;
  ">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes functions for processing and visualizing data specific to VEDA stories. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed. </div>

<h4> If you <strong>do not</strong> have a VEDA Jupyterhub Account you can launch this notebook on your local environment using MyBinder by clicking the icon below.</h4>
<br/>
<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/9c8cdbae92906fb7062b8a0c759dad90e223a4f9?urlpath=lab%2Ftree%2Fuser-guide%2Fnotebooks%2Fstories%2Fderechos.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> </a>

## Environment Setup

In [2]:
# Load libraries
#!pip install -q earthaccess pandas xarray fsspec requests pystac_client

import datetime
import glob
import os

import earthaccess
import pandas as pd
import plotutils as putils
import requests
import xarray as xr
from earthaccess import Store
from pystac_client import Client

In [3]:
# For retrieving data already catalogued in VEDA STAC
STAC_API_URL = "https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

## Overview

The Spring 2024 tornado season was one of the most active on record in the United States. As of May 31st, there were 1,176 tornadoes confirmed in the United States. The vast majority of these occurred in the meteorological spring months of March-May, placing this period within the 90th percentile of activity compared to climatological averages. This three-month period was responsible for 36 fatalities, hundreds of injuries, and was the second most prolific tornado season since at least 1950–second only to the deadly 2011 tornado season. Through the end of May, severe thunderstorms alone accounted for nearly 42 billion dollars (USD) in damage across the United States, with four tornado outbreaks during the spring attributing over $1 billion in damage each. Four tornadoes were rated violent EF-4s on the Enhanced Fujita Scale with wind speeds from 166-200 mph, and 27 were rated as intense EF-3s with wind speeds from 136-165 mph.

The National Weather Service (NWS) issued 1,728 tornado warnings across the United States from March to May, with several of these being upgraded to Particularly Dangerous Situation (PDS) Tornado Warnings if confirmed to be large and actively damaging structures. An even smaller subset was upgraded further from PDS to Tornado Emergencies if the tornado was confirmed to be large and was directly threatening a larger population center. As a result, millions of Americans were impacted by tornado warnings during the Spring of 2024, with the vast majority of counties in the Midwest, Southeast, and Southern Great Plains seeing a warning at least once during the three-month period. Compared to average spring severe weather seasons, the Southeast United States saw a less active season, while the Great Plains and Midwest saw a more active season.

# Example: NWS DAT Tornado Tracks (March-May 2024)


This example pulls tornado paths from March-May 2024 via the VEDA STAC catalog and visualize. These tornado tracks were rasterized from the NWS Damage Assessment Toolkit (DAT), which is a geographic information system (GIS)-hosted dataset where post-storm damage survey results are uploaded and referenced to the location where each description of damage occurred. This dataset includes center lines of all tornado tracks, polygons that break down the EF rating at each point along the path, descriptions of the damage at each survey location, and ground damage pictures of the damage at most entry points.


## Processing steps:
1.) Choose STAC catalog ID and date<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

## Choose variable and retrieve json from VEDA STAC catalogue

In [4]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "tornadoes-2024-paths"
date = "2024-05-31"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
#((vmin, vmax),) = dashboard_render["rescale"]
((vmin, vmax),) = ((0,5),)

collection

<CollectionClient id=tornadoes-2024-paths>

In [5]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
#colormap_name = "tornado_ef_scale"
colormap_name = "turbo"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/tornadoes-2024-paths/items/Tornado_Tracks_cog_2024-05-31/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=turbo&rescale=0%2C5'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-103.501720443, 18.467731627, -66.701720443, 46.592731627], 'center': [-85.101720443, 32.530231627, 0]}


## Plot data

In [6]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41.31, -94.46],
    zoom_level=9,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Tornado Tracks",
#    date=f"{date}T00:00:00Z",
    date="(March-May 2024)",
    colorbar_caption="EF Rating",
    attribution="VEDA - NWS DAT Tornado Tracks",
    tile_name="Tornado Tracks (March-May 2024)",
    opacity=0.8,
    height="800px",
)

# Display the map
m

# Example: NWS Tornado Polygons (March-May 2024)

Pull the NWS Tornado Polygon data from the VEDA STAC catalog and visualize


In [7]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "tornadoes-2024-polygons"
date = "2024-05-31"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
#((vmin, vmax),) = dashboard_render["rescale"]
((vmin, vmax),) = ((0,5),)

collection

<CollectionClient id=tornadoes-2024-polygons>

In [8]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
#colormap_name = "tornado_ef_scale"
colormap_name = "turbo"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/tornadoes-2024-polygons/items/Tornado_Polygons_2024-05-31_cog/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=turbo&rescale=0%2C5'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-104.72280125674247, 27.9971496536041, -71.9717534735214, 46.847529413454325], 'center': [-88.34727736513193, 37.42233953352921, 0]}


In [9]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[40.30, -84.05],
    zoom_level=10.25,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Tornado Tracks [Polygons]",
#    date=f"{date}T00:00:00Z",
    date="(March-May 2024)",
    colorbar_caption="EF Rating",
    attribution="VEDA - NWS DAT Tornado Polygons",
    tile_name="Tornado Tracks [Polygons] (March-May 2024)",
    opacity=0.8,
    height="800px",
)

# Display the map
m

# Example: Black Marble Nightlights

Pull the NASA Black Marble Nightlights data from from the VEDA STAC catalog and visualize


NASA’s Black Marble night lights dataset provides satellite images of Earth at night, capturing the light pollution from cities, roads, and other human activity. These images are collected by the Visible Infrared Imaging Radiometer Suite (VIIRS) sensor on the Suomi National Polar-orbiting Partnership (NPP) satellite, and can be used to examine changes in illumination over time. In this story, night lights data is used to assess changes before and after a tornado impacts a town, highlighting areas affected by power outages that may cause recovery slowdowns.


In [10]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "lakeview-nightlights-tornadoes-2024"
date = "2024-03-14"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=lakeview-nightlights-tornadoes-2024>

In [11]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
#colormap_name = "tornado_ef_scale"
colormap_name = "bwr"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/lakeview-nightlights-tornadoes-2024/items/nightlights_LakeviewOH_diff_cog_2024-03-14/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=bwr&rescale=-150%2C150'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-84.2954, 40.20127368220017, -83.57612536287245, 40.69122631779985], 'center': [-83.93576268143622, 40.446250000000006, 0]}


In [12]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[40.496, -83.884],
    zoom_level=12.5,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Black Marble Nightlights",
    date=f"{date}T00:00:00Z",
    colorbar_caption="Artificial Light",
    attribution="VEDA Black Marble Nightlights",
    tile_name="Black Marble Nightlights",
    opacity=0.8,
    height="800px",
)

print(
    "Visualization of Indian Lake, Ohio showing large decreases in artificial light emissions after an EF-3 tornado on March 14, 2024."
)
# Display the map
m

Visualization of Indian Lake, Ohio showing large decreases in artificial light emissions after an EF-3 tornado on March 14, 2024.


# Example: DOW7 Base Reflectivity (Radar)

Pull the DOW7 radar data from from the VEDA STAC catalog and visualize


The Doppler on Wheels (DOW) radar, which is mounted on vehicles and can get closer to storms, can be examined in cases where it was deployed in order to gather more detailed information than the NWS regional WSR-88D radars. DOW radars are typically used for research and can capture clearer, higher-resolution data on tornadoes, while WSR-88D provides a wider view but with slightly less detail.



In [13]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "tornadoes-2024-dow-refl-harlan"
date = "2024-04-26"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=tornadoes-2024-dow-refl-harlan>

In [14]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "turbo"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/tornadoes-2024-dow-refl-harlan/items/DOW_Harlan_REFL_cog_v2_2024-04-26/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=turbo&rescale=-10%2C50'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-95.42, 41.62, -95.3, 41.72], 'center': [-95.36, 41.67, 0]}


In [15]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41.668, -95.372],
    zoom_level=14,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="DOW7 Reflectivity (Harlan, IA)",
    date=f"{date}T00:00:00Z",
    colorbar_caption="dBZ",
    attribution="DOW7 Reflectivity (Harlan, IA)",
    tile_name="DOW7 Reflectivity (Harlan, IA)",
    opacity=0.8,
    height="800px",
)

print(
    "DOW-collected reflectivity values from the Minden-Harlan, Iowa EF-3 tornado on April 26, 2024 as it passed northwest of Harlan."
)
# Display the map
m

DOW-collected reflectivity values from the Minden-Harlan, Iowa EF-3 tornado on April 26, 2024 as it passed northwest of Harlan.


# Example: DOW7 Velocity (Radar)

Pull the DOW7 radar data from from the VEDA STAC catalog and visualize


In [17]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "tornadoes-2024-dow-v-harlan"
date = "2024-04-26"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=tornadoes-2024-dow-v-harlan>

In [18]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "bwr"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/tornadoes-2024-dow-v-harlan/items/DOW_Harlan_V_cog_v2_2024-04-26/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=bwr&rescale=-75%2C75'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-95.42, 41.62, -95.3, 41.72], 'center': [-95.36, 41.67, 0]}


In [19]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41.668, -95.372],
    zoom_level=14,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="DOW7 Velocity (Harlan, IA)",
    date=f"{date}T00:00:00Z",
    colorbar_caption="m/s",
    attribution="DOW7 Velocity (Harlan, IA)",
    tile_name="DOW7 Velocity (Harlan, IA)",
    opacity=0.8,
    height="800px",
)

print(
    "DOW-collected velocity values from the Minden-Harlan, Iowa EF-3 tornado on April 26, 2024 as it passed northwest of Harlan."
)
# Display the map
m

DOW-collected velocity values from the Minden-Harlan, Iowa EF-3 tornado on April 26, 2024 as it passed northwest of Harlan.


# Example: DOW7 Correlation Coefficient (Radar)

Pull the DOW7 radar data from from the VEDA STAC catalog and visualize


In [21]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "tornadoes-2024-dow-rhohv-greenfield"
date = "2024-05-21"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=tornadoes-2024-dow-rhohv-greenfield>

In [22]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "turbo"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/tornadoes-2024-dow-rhohv-greenfield/items/DOW_Greenfield_RHOHV_cog_v2_2024-05-21/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=turbo&rescale=0%2C1'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-94.5, 41.275, -94.4, 41.325], 'center': [-94.45, 41.3, 0]}


In [26]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41.3036, -94.4569],
    zoom_level=16,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="DOW7 Correlation Coefficient (Greenfield, IA)",
    date=f"{date}T00:00:00Z",
    attribution="DOW7 Correlation Coefficient (Greenfield, IA)",
    tile_name="DOW7 Correlation Coefficient (Greenfield, IA)",
    opacity=0.8,
    height="800px",
)

print(
    "DOW-collected correlation coefficient values from the Greenfield, Iowa EF-4 tornado on May 21, 2024 as it was impacting Greenfield."
)
# Display the map
m

DOW-collected correlation coefficient values from the Greenfield, Iowa EF-4 tornado on May 21, 2024 as it was impacting Greenfield.


# Example: DOW7-Derived Maximum Velocity Swath (Radar)

Pull the DOW7 radar data from from the VEDA STAC catalog and visualize


In [28]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "tornadoes-2024-dow-vmax-greenfield"
date = "2024-05-21"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=tornadoes-2024-dow-vmax-greenfield>

In [29]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "gist_ncar"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/tornadoes-2024-dow-vmax-greenfield/items/DOW_Vmax_Greenfield_mph_cog_2024-05-21/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=gist_ncar&rescale=55%2C250'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-94.4784, 41.29080000000008, -94.416, 41.31980000000008], 'center': [-94.4472, 41.30530000000008, 0]}


In [30]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41.3036, -94.4569],
    zoom_level=16,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="DOW7-Derived Maximum Velocity (Greenfield, IA)",
    date=f"{date}T00:00:00Z",
    colorbar_caption="m/s",
    attribution="DOW7-Derived Maximum Velocity  (Greenfield, IA)",
    tile_name="DOW7-Derived Maximum Velocity (Greenfield, IA)",
    opacity=0.8,
    height="800px",
)

print(
    "DOW-collected correlation coefficient values from the Greenfield, Iowa EF-4 tornado on May 21, 2024 as it was impacting Greenfield."
)
# Display the map
m

DOW-collected correlation coefficient values from the Greenfield, Iowa EF-4 tornado on May 21, 2024 as it was impacting Greenfield.


# Clean Up (Optional)


Remove any **.gif** files that were created to save on storage space. Additionally, remove any **core** files that were created if the kernel crashed.

In [31]:
# find all .gif files in the current directory
for gif_path in glob.glob("*.gif"):
    try:
        os.remove(gif_path)
        print(f"Removed {gif_path}")
    except OSError as e:
        print(f"Error removing {gif_path}: {e}")

# find all core files in the current directory
for core_path in glob.glob("core.*"):
    try:
        os.remove(core_path)
        print(f"Removed {core_path}")
    except OSError as e:
        print(f"Error removing {core_path}: {e}")